In [7]:
import VariationalBayes as vb
import LogisticGLMM_lib as logit_glmm
import VariationalBayes.SparseObjectives as obj_lib
import VariationalBayes.ExponentialFamilies as ef

import matplotlib.pyplot as plt
%matplotlib inline

import autograd
import numpy as np
import scipy as sp
from scipy import sparse

import copy
from scipy import optimize

import os
import json

import time

import pickle

from scikits.sparse.cholmod import cholesky


analysis_name = 'simulated_data_small'
#analysis_name = 'criteo_subsampled'

data_dir = os.path.join(os.environ['GIT_REPO_LOC'],
                        'LinearResponseVariationalBayes.py/Models/LogisticGLMM/data')


In [11]:
pickle_output_filename = os.path.join(data_dir, '%s_python_vb_results.pkl' % analysis_name)
pkl_file = open(pickle_output_filename, 'rb')
vb_data = pickle.load(pkl_file)
model = logit_glmm.load_model_from_pickle(vb_data)
timer = vb_sparse.Timer()
kl_hess = obj_lib.unpack_csr_matrix(vb_data['kl_hess_packed'])
moment_jac = vb_data['moment_jac']
moment_wrapper = logit_glmm.MomentWrapper(model.glmm_par)

In [12]:
def optimize_with_weights(
    model, weights, init_par, preconditioner, gtol=1e-8, print_every=1, verbose=True):

    model.use_weights = True
    model.weights = copy.deepcopy(weights)
    preconditioned_init_par = sp.sparse.linalg.spsolve(preconditioner, init_par)
    return model.tr_optimize_cond(
        preconditioned_init_par,
        preconditioner=preconditioner,
        num_gh_points=model.num_gh_points,
        gtol=gtol,
        print_every=print_every,
        verbose=verbose)   


In [13]:
# Pre-condition with the KL Hessian from the original analysis.
preconditioner = sp.sparse.diags([1 / np.sqrt(kl_hess.diagonal())], [0])
preconditioner = sp.sparse.csr_matrix(preconditioner)
# cond_init = sp.sparse.linalg.spsolve(preconditioner, glmm_par_free)


In [17]:
base_weights = np.full(model.x_mat.shape[0], 1.0)
print(np.sum(base_weights))
base_opt = optimize_with_weights(
    model=model,
    preconditioner=preconditioner,
    weights=base_weights,
    init_par=glmm_par_free,
    gtol=1e-8)
base_free_par = model.objective.uncondition_x(base_opt.x)

1000.0
Iter  0  value:  69.4078730719
Iter  1  value:  69.4078730719
         Current function value: 69.407873
         Iterations: 0
         Function evaluations: 2
         Gradient evaluations: 1
         Hessian evaluations: 0


In [19]:
timer.tic()
moment_jac = moment_wrapper.get_moment_jacobian(base_free_par)
timer.toc('moment_jac_time')
moment_jac_sp = sp.sparse.csr_matrix(moment_jac)

moment_jac_time: 0.12421250343322754 seconds


In [20]:
# Get a sparse Jacobian.
timer.tic()
weight_jacobian = model.get_sparse_weight_free_jacobian(base_free_par, print_every_n=200)
timer.toc('weight_jac_time')
print('Jacobian time: ', timer.time_dict['weight_jac_time'])

Group 0 of 99
weight_jac_time: 1.4826602935791016 seconds
Jacobian time:  1.4826602935791016


In [21]:
# Get a sparse Hessian.
timer.tic()
kl_hess = model.get_sparse_free_hessian(base_free_par, print_every_n=200)
timer.toc('hess_time')
print('Hessian time: ', timer.time_dict['hess_time'])

Group 0 of 99.
hess_time: 2.3268980979919434 seconds
Hessian time:  2.3268980979919434


/home/rgiordan/Documents/git_repos/autograd/autograd/core.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


In [22]:
# Invert to get parameter sensitivity.
timer.tic()
print('Cholesky...')
kl_hess_chol = cholesky(kl_hess)
print('Solving...')
param_boot_mat = -1 * kl_hess_chol.solve_A(weight_jacobian.T)
print('Done.')
timer.toc('inverse_time')

Cholesky...
Solving...
Done.
inverse_time: 0.011031866073608398 seconds


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  after removing the cwd from sys.path.


In [23]:
def bootstrap_and_refit(model, preconditioner, param_boot_mat, base_free_par):
    num_obs = model.x_mat.shape[0]
    boot_draw = np.random.multinomial(num_obs, [1. / num_obs] * num_obs, size=1)
    boot_draw = np.asarray(np.squeeze(boot_draw))
    lr_param_diff = param_boot_mat * (boot_draw - 1.0)
    glmm_par_lr_boot = base_free_par + lr_param_diff
    glmm_par_opt_boot = optimize_with_weights(
        model=model,
        preconditioner=preconditioner,
        init_par=glmm_par_lr_boot,
        weights=boot_draw,
        gtol=1e-8,
        verbose=False)
    boot_free_par = model.objective.uncondition_x(glmm_par_opt_boot.x)
    return boot_free_par, glmm_par_opt_boot, glmm_par_lr_boot

In [ ]:
num_boot_samples = 50
lr_boot_free_par_list = []
boot_free_par_list = []
timer.tic()
for boot in range(num_boot_samples):
    if boot % 20 == 0:
        print('Bootstrap {} of {}'.format(boot + 1, num_boot_samples))
    boot_free_par, glmm_par_opt_boot, glmm_par_lr_boot = \
        bootstrap_and_refit(model, preconditioner, param_boot_mat, base_free_par)
    boot_free_par_list.append(boot_free_par)
    lr_boot_free_par_list.append(glmm_par_lr_boot)
timer.toc('bootstrap_time')

Bootstrap 1 of 50


In [ ]:
# Evaluated at a linear combination of the bootstrap draw and base weights.
use_jackknife = False
if use_jackknife:
    boot_draw = copy.deepcopy(base_weights)
    boot_draw[np.random.randint(NObs)] = 0.
else:
    boot_draw = np.random.multinomial(NObs, [1. / NObs] * NObs, size=1) - 1.0
    boot_draw = 1.0 * boot_draw + base_weights

print('Total weight (there are {} observations): {}'.format(NObs, np.sum(boot_draw)))
lr_boot_time = time.time()
lr_param_diff = param_boot_mat * (np.squeeze(boot_draw) - 1.0)
lr_boot_time = time.time() - lr_boot_time

print('lr_boot_time: ', lr_boot_time)


In [ ]:
#preconditioner = sp.sparse.diags([1 / np.sqrt(kl_hess.diagonal())], [0])
# cond_init = sp.sparse.linalg.spsolve(preconditioner, base_free_par + lr_param_diff)

timer.tic()
glmm_par_opt_boot = optimize_with_weights(
    model=model,
    preconditioner=preconditioner,
    init_par=base_free_par + lr_param_diff,
    weights=boot_draw, gtol=1e-8)
boot_free_par = model.objective.uncondition_x(glmm_par_opt_boot.x)
timer.toc('re_optimization_time')

print('Re-optimization time: ', timer.time_dict['re_optimization_time'])

In [ ]:
# The actual change in the moment vectors.
moment_vec = moment_wrapper.get_moment_vector_from_free(base_free_par)
moment_vec_boot = moment_wrapper.get_moment_vector_from_free(boot_free_par)
true_moment_diff = moment_vec_boot - moment_vec
print('True param difference: {}'.format(np.linalg.norm(base_free_par - boot_free_par)))
print('True moment norm difference: {}'.format(np.linalg.norm(true_moment_diff)))

# Use the linear approximation for the parameters, not the moments.
moment_pred_from_params = \
    moment_wrapper.get_moment_vector_from_free(base_free_par + lr_param_diff)
moment_pred_from_params_diff = moment_pred_from_params - moment_vec

# The difference based on a linear approximation to the moments.
# lr_moment_diff = \
#     moment_jac_sp * (param_boot_mat * (np.squeeze(boot_draw) - 1.0))
moment_pred_from_moments_diff = moment_jac_sp * lr_param_diff
    

plt.figure()
plt.title('Linearizing the moments')
plt.plot(true_moment_diff, moment_pred_from_moments_diff, 'r+', markersize=10)
plt.plot(true_moment_diff, true_moment_diff, 'k.')
#plt.axis([-10, 10, -10, 10])

plt.figure()
plt.title('Linearizing the parameters')
plt.plot(true_moment_diff, moment_pred_from_params_diff, 'bx', markersize=10)
plt.plot(true_moment_diff, true_moment_diff, 'k.')
#plt.axis([-10, 10, -10, 10])

plt.figure()
plt.title('Moments vs parameters')
plt.plot(moment_pred_from_params_diff, moment_pred_from_moments_diff, 'r+', markersize=10)
plt.plot(moment_pred_from_params_diff, moment_pred_from_params_diff, 'k.')
#plt.axis([-10, 10, -10, 10])



In [ ]:
# Write the result to a pickle file for use in R.

run_name = 'jackknife'

pickle_result_filename = os.path.join(data_dir, '%s_python_vb_jackknife_results.pkl' % analysis_name)
pickle_output = open(pickle_result_filename, 'wb')

# Unlike with JSON, numpy arrays can be pickled.
pickle_result_dict = {  'boot_free_par_list': boot_free_par_list,
                        'lr_boot_free_par_list': lr_boot_free_par_list,
                        'run_name': run_name,
                        'beta_dim': K,
                        'num_groups': NG,
                        'num_gh_points': model.num_gh_points,
                        'timer': timer,
                        'base_free_par': base_free_par,
                        'boot_free_par': boot_free_par,
                        'boot_draw': boot_draw,
                        'kl_hess': vb_sparse.pack_csr_matrix(kl_hess),
                        'weight_jacobian': vb_sparse.pack_csr_matrix(sp.sparse.csr_matrix(weight_jacobian)),
                        'param_boot_mat': vb_sparse.pack_csr_matrix(sp.sparse.csr_matrix(param_boot_mat)),
                        'moment_jac': moment_jac }

# Pickle dictionary using protocol 0.
pickle.dump(pickle_result_dict, pickle_output)
pickle_output.close()

print(pickle_result_filename)


print('\n\nDONE.')
